        Say we have an image of 28x28 i.e. 784 features, then if we add a hidden layer with even 100 units, we will have 78400 weights and updating so many weights would be hectic and would result in overfitting. If this image is much bigger then it would be worse. So we have to avoid this, we will use a method of convolution layer
    
    Concepts Used:
        Padding
        Stride (how much gap)
        Forming Channels
        Pooling (max pooling works much better)
        Stretching (i.e. these pixels after pooling will be considered as new features and will pass through hidden layer and further)

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot = True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input = 784
n_classes = 10
learning_rate = 0.01
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

    weights of c1 has 4 values: rows, columns, depth, n of units in that layer

In [3]:
weights = {
    'c1' : tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'c2' : tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'h1' : tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out' : tf.Variable(tf.random_normal([1024, n_classes]))
}

In [4]:
biases = {
    'c1' : tf.Variable(tf.random_normal([32])),
    'c2' : tf.Variable(tf.random_normal([64])),
    'h1' : tf.Variable(tf.random_normal([1024])),
    'out' : tf.Variable(tf.random_normal([n_classes]))
}

In [5]:
def conv(x, weights, biases, strides = 1):
    x = tf.nn.conv2d(x, weights, padding = 'SAME', strides = [1, strides, strides, 1], use_cudnn_on_gpu = True)    # this is inbuilt func, but it doesn't include biases, strides has some values, whose first value is 1 and last value(depth) will be 1
    x = tf.nn.bias_add(x, biases)
    return tf.nn.relu(x)

def maxpooling(conv, k):
    return tf.nn.max_pool(conv, padding = 'SAME', ksize = [1, k, k, 1], strides = [1, k, k, 1])
    
def cnn_result(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1, 28, 28, 1])
    
    conv1 = conv(x, weights['c1'], biases['c1'])
    conv1 = maxpooling(conv1, k = 2)    # here k = pool size
    
    conv2 = conv(conv1, weights['c2'], biases['c2'])
    conv2 = maxpooling(conv2, k = 2)
    
    hidden_input = tf.reshape(conv2, shape = [-1, 7 * 7 * 64])
    hidden_output_before_relu = tf.add(tf.matmul(hidden_input, weights['h1']), biases['h1'])
    hidden_output = tf.nn.relu(hidden_output_before_relu)
    
        # dropout those outputs, 
    hidden_output = tf.nn.dropout(hidden_output, keep_prob)
    
    out_layer = tf.matmul(hidden_output, weights['out']) + biases['out']
    
    return out_layer

In [6]:
pred = cnn_result(x, weights, biases, 0.75)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [7]:
init = tf.global_variables_initializer()
batch_size = 100
sess = tf.Session()
sess.run(init)

for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _,c = sess.run([optimizer, cost], feed_dict = {x:batch_x, y:batch_y})
        total_cost += c
    print (total_cost)

933798.720995
39221.2301272
22627.1915514
15456.2256817
11103.4317173
10113.4105084
7423.04772268
6493.56750507
5275.32351841
6065.99272602
5444.98623747
3687.26005494
3681.52995338
3585.88074394
3145.10566601
3025.78951702
2433.82805677
2327.4125915
2595.47355897
2580.50582671
1776.57954914
2175.72579796
2055.79514906
2109.00953255
1327.20202589


In [8]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
predictions = sess.run(correct_prediction, feed_dict = {x:mnist.test.images, y:mnist.test.labels})
predictions.sort()
print (predictions)

[False False False ...,  True  True  True]


In [19]:
predictions[178]

False

In [20]:
print ('accuracy = ', (55000-178)/55000)

accuracy =  0.9967636363636364
